In [1]:
import numpy as np
import pandas as pd
import os
import random 
from shutil import copyfile
import pydicom as dicom
import cv2

In [2]:
savepath = '/data/'
COVIDPath = '/covid19/'
Pneumonia1 = '/Chexpert/Pneumonia/'
Pneumonia2 = '/NIH/Pneumonia/'
Pneumonia3 = '/Chexpert/Pneumonia+1comorb/'
Pneumonia4 = '/projects/ceib/padchest_covid19/pneumonia/'
SARS = '/SARS/'
ARDS = '/ARDS/'
Pneumocystis = '/Pneumocystis/'
Control1 = '/NIH/NoFinding/'
Streptococcus = '/Streptococcus/'
paths = [COVIDPath,Pneumonia1,Pneumonia2,Pneumonia3,Pneumonia4,SARS,ARDS,Pneumocystis,Streptococcus,Control1]
mapping = ['COVID-19','pneumonia','pneumonia','pneumonia','pneumonia','pneumonia','pneumonia','pneumonia','pneumonia','normal']

In [3]:
# parameters for COVIDx dataset
train = []
test = []
test_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
path_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}

In [4]:
for i,directories in enumerate(paths):
    arr = os.listdir(directories + '.')
    for j,file in enumerate(arr):
        count[mapping[i]] += 1
        entry = [int(count[mapping[i]]), file, mapping[i]]
        filename_label[mapping[i]].append(entry)
        path_label[mapping[i]].append(directories)

In [5]:
np.random.seed(seed=2)
test_per= 0.1
for key in filename_label.keys():
    arr = np.array(filename_label[key])
    imgpath = np.array(path_label[key])
    if arr.size == 0:
        continue
    if key == 'pneumonia':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]#['8', '31']
    elif key == 'COVID-19':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]#['19', '20', '36', '42', '86'] # random.sample(list(arr[:,0]), num_test)
    else: 
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]#[]
    #print('Key: ', key)
    #print('Test patients: ', test_patients)
    # go through all the patients
    for i,patient in enumerate(arr):
        namesave = patient[1].replace(" ", "_")
        if int(patient[0]) in test_patients:
            copyfile(os.path.join(imgpath[i], patient[1]), os.path.join(savepath, 'test', namesave))
            patient[1] = namesave
            test.append(patient)
            test_count[patient[2]] += 1
        else:
            copyfile(os.path.join(imgpath[i], patient[1]), os.path.join(savepath, 'train', namesave))
            patient[1] = namesave
            if key == 'COVID-19':
                train.append(patient)
                train.append(patient)#Data augmentation
                train.append(patient)#Data augmentation
            elif key == 'pneumonia':
                train.append(patient)
                train.append(patient)
                train.append(patient)#Data augmentation
            else:
                train.append(patient)
            train_count[patient[2]] += 1

print('test count: ', test_count)
print('train count: ', train_count)

test count:  {'normal': 2129, 'pneumonia': 120, 'COVID-19': 38}
train count:  {'normal': 19166, 'pneumonia': 1089, 'COVID-19': 348}


In [6]:
# path to https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
kaggle_datapath = '/home/julian/Documents/Datasets/rsna-pneumonia-detection-challenge'
kaggle_csvname = '/home/julian/Documents/Datasets/rsna-pneumonia-detection-challenge/stage_2_detailed_class_info.csv' # get all the normal from here
kaggle_csvname2 = '/home/julian/Documents/Datasets/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv' # get all the 1s from here since 1 indicate pneumonia
kaggle_imgpath = '/home/julian/Documents/Datasets/rsna-pneumonia-detection-challenge/stage_2_train_images'

# add normal and rest of pneumonia cases from https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
csv_normal = pd.read_csv(os.path.join(kaggle_datapath, kaggle_csvname), nrows=None)
csv_pneu = pd.read_csv(os.path.join(kaggle_datapath, kaggle_csvname2), nrows=None)
patients = {'normal': [], 'pneumonia': []}

for index, row in csv_normal.iterrows():
    if row['class'] == 'Normal':
        patients['normal'].append(row['patientId'])

for index, row in csv_pneu.iterrows():
    if int(row['Target']) == 1:
        patients['pneumonia'].append(row['patientId'])

for key in patients.keys():
    if key == 'normal':
        arr = np.array(patients[key])
        if arr.size == 0:
            continue
        # split by patients 
        # num_diff_patients = len(np.unique(arr))
        # num_test = max(1, round(split*num_diff_patients))
        test_patients = []#np.load('rsna_test_patients_{}.npy'.format(key)) # random.sample(list(arr), num_test), download the .npy files from the repo.
        # np.save('rsna_test_patients_{}.npy'.format(key), np.array(test_patients))
        for patient in arr:
            ds = dicom.dcmread(os.path.join(kaggle_datapath, kaggle_imgpath, patient + '.dcm'))
            pixel_array_numpy = ds.pixel_array
            imgname = patient + '.png'
            if patient in test_patients:
                cv2.imwrite(os.path.join(savepath, 'test', imgname), pixel_array_numpy)
                test.append([patient, imgname, key])
                test_count[key] += 1
            else:
                cv2.imwrite(os.path.join(savepath, 'train', imgname), pixel_array_numpy)
                train.append([patient, imgname, key])
                train_count[key] += 1

print('test count: ', test_count)
print('train count: ', train_count)

test count:  {'normal': 2535, 'pneumonia': 116, 'COVID-19': 38}
train count:  {'normal': 31671, 'pneumonia': 1052, 'COVID-19': 348}


In [6]:
train_file = open("train_split_v2.txt","a") 
for sample in train:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    train_file.write(info)

train_file.close()

test_file = open("test_split_v2.txt", "a")
for sample in test:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    test_file.write(info)

test_file.close()